In [1]:
library(tidyverse)
library(cytominer)
library(magrittr)

── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 3.0.0     ✔ purrr   0.2.5
✔ tibble  1.4.2     ✔ dplyr   0.7.6
✔ tidyr   0.8.1     ✔ stringr 1.3.1
✔ readr   1.1.1     ✔ forcats 0.3.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Attaching package: ‘cytominer’

The following object is masked from ‘package:stats’:

    aggregate

The following object is masked from ‘package:base’:

    transform


Attaching package: ‘magrittr’

The following object is masked from ‘package:purrr’:

    set_names

The following object is masked from ‘package:tidyr’:

    extract



# function to load different data sets
This function also adds the Metdata columns Metadata_dataset, Metadata_partition and Metadata_features

In [2]:
load_dataset  <- function(partition, dataset,feature){
    file_name  <- read_csv("../datasets.csv") 
    x  <-  file_name %>% filter(
         Partition == partition,
         Dataset == dataset,
         Features == feature) %>% 
         extract2("Link")

    return(read_csv(x) %>% 
          mutate(Metadata_dataset = dataset) %>%
          mutate(Metadata_partition = partition) %>% 
          mutate(Metadata_features = feature) 
          )
    }

# Load data 
We load training and test datasets for both BBBC036 / CDRP data set and select only important Metadata colums

In [3]:
# bbbc36 data 
bbbc036_train  <- load_dataset("Train","BBBC036","CellProfiler")  %>% 
    mutate(Metadata_x_mutation_status = "none")

bbbc036_test <- load_dataset("Test","BBBC036","CellProfiler")  %>% 
    mutate(Metadata_x_mutation_status = "none")


Parsed with column specification:
cols(
  Dataset = col_character(),
  Partition = col_character(),
  Features = col_character(),
  Link = col_character()
)
Parsed with column specification:
cols(
  .default = col_double(),
  Metadata_Plate = col_integer(),
  Metadata_Well = col_character(),
  Metadata_Assay_Plate_Barcode = col_integer(),
  Metadata_Plate_Map_Name = col_character(),
  Metadata_well_position = col_character(),
  Metadata_ASSAY_WELL_ROLE = col_character(),
  Metadata_broad_sample = col_character(),
  Metadata_solvent = col_character(),
  Metadata_pert_id = col_character(),
  Metadata_pert_mfc_id = col_character(),
  Metadata_pert_well = col_character(),
  Metadata_pert_id_vendor = col_character(),
  Metadata_cell_id = col_character(),
  Metadata_broad_sample_type = col_character(),
  Metadata_pert_vehicle = col_character(),
  Metadata_pert_type = col_character(),
  Cells_AreaShape_EulerNumber = col_integer(),
  Cells_Children_Cytoplasm_Count = col_integer(),
  Cells_Neig

In [4]:
bbbc036  <- rbind(bbbc036_train, bbbc036_test)  

# How large are the data sets? 

In [5]:
bbbc036_train %>% dim()
bbbc036_test %>% dim()

[1] 18929  1805

[1] 2177 1805

# What are the Metadata colums?

In [6]:
bbbc036 %>% 
    select(Metadata_Plate, Metadata_Well, Metadata_Plate_Map_Name, Metadata_pert_id) %>% 
    slice(1:5) %>% 
    print()

# A tibble: 5 x 4
  Metadata_Plate Metadata_Well Metadata_Plate_Map_Name Metadata_pert_id
           <int> <chr>         <chr>                   <chr>           
1          24277 a01           H-BIOA-004-3            BRD-K18250272   
2          24277 a02           H-BIOA-004-3            BRD-K18316707   
3          24277 a03           H-BIOA-004-3            BRD-K18438502   
4          24277 a04           H-BIOA-004-3            BRD-K18550767   
5          24277 a05           H-BIOA-004-3            BRD-K18574842   


In [27]:
bbbc036 %>% 
    select( Metadata_pert_id, Metadata_broad_sample_type,Metadata_dataset, Metadata_partition) %>% 
    slice(1:5) %>% 
    print()

# A tibble: 5 x 4
  Metadata_pert_id Metadata_broad_sample_ty… Metadata_dataset Metadata_partiti…
  <chr>            <chr>                     <chr>            <chr>            
1 BRD-K18250272    trt                       BBBC036          Train            
2 BRD-K18316707    trt                       BBBC036          Train            
3 BRD-K18438502    trt                       BBBC036          Train            
4 BRD-K18550767    trt                       BBBC036          Train            
5 BRD-K18574842    trt                       BBBC036          Train            


# how many replicates do we have? 

In [8]:
bbbc036 %>% 
    group_by(Metadata_Plate_Map_Name, Metadata_pert_id) %>%
    summarise(n_groups = n()) %>%
    summarise(mean_replicates = mean(n_groups))

Metadata_Plate_Map_Name,mean_replicates
H-BIOA-001-3,9.600000
H-BIOA-002-3,8.373832
H-BIOA-003-3,9.554517
H-BIOA-004-3,9.570093
H-BIOA-005-3,9.563863
H-BIOA-006-3,9.554517
H-BIOA-007-3,9.593750


In [9]:
# bbbc022 data 
bbbc022_train  <- load_dataset("Train","BBBC022","CellProfiler")
 
bbbc022_test <- load_dataset("Test","BBBC022","CellProfiler")

bbbc022  <- rbind(bbbc022_train, bbbc022_test) %>%
  select(Metadata_Plate, Metadata_Well, Metadata_Plate_Map_Name, 
         Metadata_pert_id, Metadata_broad_sample_type,
         Metadata_dataset,Metadata_partition, 
         everything()
        ) 

Parsed with column specification:
cols(
  Dataset = col_character(),
  Partition = col_character(),
  Features = col_character(),
  Link = col_character()
)
Parsed with column specification:
cols(
  .default = col_double(),
  Metadata_Plate = col_integer(),
  Metadata_Well = col_character(),
  Metadata_Assay_Plate_Barcode = col_integer(),
  Metadata_Plate_Map_Name = col_character(),
  Metadata_well_position = col_character(),
  Metadata_broad_sample = col_character(),
  Metadata_source_name = col_character(),
  Metadata_compound_name = col_character(),
  Metadata_smiles = col_character(),
  Metadata_solvent = col_character(),
  Metadata_pert_id = col_character(),
  Metadata_pert_mfc_id = col_character(),
  Metadata_pert_well = col_character(),
  Metadata_pert_id_vendor = col_character(),
  Metadata_cell_id = col_character(),
  Metadata_broad_sample_type = col_character(),
  Metadata_pert_vehicle = col_character(),
  Metadata_pert_type = col_character(),
  Metadata_exp = col_character()

# How large are the training and test partitions for BBBC022?

In [10]:
bbbc022_train %>% dim()
bbbc022_test %>% dim()

[1] 6462 1806

[1] 1120 1806

In [11]:
bbbc022 %>% 
    group_by(Metadata_Plate_Map_Name, Metadata_pert_id) %>%
    summarise(n_groups = n()) %>%
    summarise(mean_replicates = mean(n_groups))

Metadata_Plate_Map_Name,mean_replicates
H-BIOA-002-1,4.860759
H-BIOA-003-1,4.780255
H-BIOA-004-1,4.830128
H-BIOA-005-1,4.816456
H-BIOA-006-1,4.797468


# How large are the combined data sets? 

In [12]:
dim(bbbc022)
dim(bbbc036)

[1] 7582 1806

[1] 21106  1805

## Extract common features and common metadata

In [13]:
colnames_bbbc022 <- colnames(bbbc022)
colnames_bbbc036 <- colnames(bbbc036)

Metadata_names_bbbc022 <- c(
   stringr::str_subset(colnames_bbbc022, "^Meta")
) 

Metadata_names_bbbc036 <- c(
   stringr::str_subset(colnames_bbbc036, "^Meta")
) 

common_metadata  <- intersect(Metadata_names_bbbc022, Metadata_names_bbbc036)  
common_features  <- setdiff(intersect(colnames_bbbc022, colnames_bbbc036),common_metadata)

# Concatenate data sets

In [14]:
population  <- rbind(
    bbbc022 %>% 
        select(c(common_metadata, common_features)),
    bbbc036 %>% 
        select(c(common_metadata, common_features))
    ) %>% 
    mutate(Metadata_perturbation = 'chemical') %>% 
    select(Metadata_perturbation, everything())

## Important: update column names! 

In [15]:
colnames_combined  <- colnames(population)

common_metadata  <- c(
   stringr::str_subset(colnames_combined, "^Meta")
) 

common_features  <- setdiff(colnames_combined, common_metadata)

Cytominer has problems handling column names '1', '2' so we rename them to 'Feature_1', ... 

In [16]:
common_features  <- paste0("Feature_",common_features)
colnames(population)  <- c(common_metadata, common_features)

# Normalize data
We use cytominer to normalize both datasets with respect to the controls, i.e. EMPTY genes

In [17]:
population_normalized  <- cytominer::normalize(
    population, 
    variables = common_features, 
    strata = c("Metadata_perturbation"), 
    sample = population %>% 
                filter(
                    Metadata_broad_sample == "DMSO",
                    Metadata_partition == "Train"
                ), 
    operation = "standardize"
)

# Aggregate data 

In [18]:
population_aggregated  <- cytominer::aggregate(
    population = population_normalized, 
    variables = common_features, 
    strata = c("Metadata_pert_id","Metadata_dataset","Metadata_partition"), 
    operation = "mean"
) 

population_aggregated %>% dim()

[1] 3782 1786

In [19]:
population_aggregated %>% filter(Metadata_dataset == "BBBC022") %>% dim()
population_aggregated %>% filter(Metadata_dataset == "BBBC036") %>% dim()

[1] 1543 1786

[1] 2239 1786

# Correlation matrix 

In [20]:
cor_matrix  <- cor(
    x = population_aggregated %>% 
        filter(Metadata_dataset == 'BBBC022') %>% 
        select(common_features) %>% 
        as.matrix() %>% 
        t, 
    y = population_aggregated %>% 
        filter(Metadata_dataset == 'BBBC036') %>% 
        select(common_features) %>% 
        as.matrix() %>% 
        t,
    use  = "complete.obs"
    ) 

cor_matrix %>% dim()

[1] 1543 2239

# Submision file 

In [21]:
# set column names 
colnames(cor_matrix)  <- population_aggregated %>% 
                            filter(Metadata_dataset == 'BBBC036') %>%
                            extract2("Metadata_pert_id")

# set row names 
#rownames(cor_matrix)  <- population_aggregated %>% 
#                            filter(Metadata_dataset == 'BBBC036') %>%
#                            extract2("Metadata_broad_sample")#

cor_matrix %>% dim()

[1] 1543 2239

In [23]:
df  <- cor_matrix %>% as_data_frame() %>% 
            mutate(Metadata_pert_id = population_aggregated %>% 
                            filter(Metadata_dataset == 'BBBC022') %>%
                            extract2("Metadata_pert_id")) %>% 
            select(Metadata_pert_id, everything())

# write submission file
write.csv(df,"../cytodata-baseline_R_day_2_CP.csv",row.names = FALSE)

In [25]:
dim(df)

[1] 1543 2240